A user has requested how many seven day prescriptions there are for Aspirin as a proprotion of all prescriptions for Aspirin. The aim is to use as a surrogate for dosette boxes.

Aspirin 75mg is once daily so prescriptions with a seven day course length are indicative of weekly scripts which are sometimes used (erroneously)  to support dispensing in dosette boxes. 

In [3]:
##importing libraries
import pandas as pd
import numpy as np
from ebmdatalab import bq, maps, charts
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

In [2]:
sql = '''
SELECT
  PCO_Name,
  PCO_Code,
  bnf_code AS bnf_code,
  bnf_description AS bnf_name,
  Items,
  Quantity
FROM
  ebmdatalab.tmp_eu.raw_prescribing_data_2019_04 AS presc
WHERE
  (bnf_code LIKE "0209000A0AA%BAB" ##dispersible 75mg
  OR bnf_code LIKE "0209000A0AA%JAJ" ##normal 75mg tabs
  OR bnf_code LIKE "0209000A0AA%KAK") ##enteric coated 75mg
'''

df_aspirin = bq.cached_read(sql, csv_path='aspirin_sevendays.csv')
df_aspirin.rename(columns={'PCO_Code':'pct'}, inplace=True)
df_aspirin.head(5)

,PCO_Name,pct,bnf_code,bnf_name,Items,Quantity
0,SOUTH WARWICKSHIRE CCG,05R00,0209000A0AAABAB,Aspirin Disper_Tab 75mg,98,28
1,SOUTH WARWICKSHIRE CCG,05R00,0209000A0AAABAB,Aspirin Disper_Tab 75mg,19,56
2,SOUTH WARWICKSHIRE CCG,05R00,0209000A0AAABAB,Aspirin Disper_Tab 75mg,15,7
3,SOUTH WARWICKSHIRE CCG,05R00,0209000A0AAABAB,Aspirin Disper_Tab 75mg,262,28
4,SOUTH WARWICKSHIRE CCG,05R00,0209000A0AAABAB,Aspirin Disper_Tab 75mg,5,56


In [3]:
##check Englaish total 
eng_total = df_aspirin.sum()[['Items','Quantity']]
eng_total

Items       1830321
Quantity    3137207
dtype: object

In [4]:
##group and sum items
ccg_total = df_aspirin.groupby('PCO_Name').sum()[['Items']]
ccg_total.head(5)

,Items
PCO_Name,
"AIREDALE, WHARFEDALE AND CRAVEN CCG",5081
ASHFORD CCG,3521
BARKING & DAGENHAM CCG,6031
BARNET CCG,9132
BARNSLEY CCG,14304


In [5]:
##islolate prescriptions with quantities of 7
df_aspirin_sevendays = df_aspirin.loc[df_aspirin['Quantity'] == 7]
df_aspirin_sevendays.rename(columns={'Items':'sevendayitems'}, inplace=True) ##renaming column
df_aspirin_sevendays.head(5)

C:\Users\bmackenna\Anaconda3\lib\site-packages\pandas\core\frame.py:4025: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


,PCO_Name,pct,bnf_code,bnf_name,sevendayitems,Quantity
2,SOUTH WARWICKSHIRE CCG,05R00,0209000A0AAABAB,Aspirin Disper_Tab 75mg,15,7
7,SOUTH WARWICKSHIRE CCG,05R00,0209000A0AAABAB,Aspirin Disper_Tab 75mg,50,7
11,SOUTH WARWICKSHIRE CCG,05R00,0209000A0AAABAB,Aspirin Disper_Tab 75mg,47,7
14,SOUTH WARWICKSHIRE CCG,05R00,0209000A0AAABAB,Aspirin Disper_Tab 75mg,5,7
18,SOUTH WARWICKSHIRE CCG,05R00,0209000A0AAABAB,Aspirin Disper_Tab 75mg,80,7


In [6]:
ccg_sevenday_total = df_aspirin_sevendays.groupby('PCO_Name').sum()[['sevendayitems']]
ccg_sevenday_total.head(5)

,sevendayitems
PCO_Name,
"AIREDALE, WHARFEDALE AND CRAVEN CCG",600
ASHFORD CCG,119
BARKING & DAGENHAM CCG,2999
BARNET CCG,3235
BARNSLEY CCG,4509


In [7]:
dosette_boxes_ccg = pd.concat([ccg_total, ccg_sevenday_total], axis=1, join='inner')
dosette_boxes_ccg["percent_seven_days"] = 100*(dosette_boxes_ccg.sevendayitems/dosette_boxes_ccg.Items).fillna(0)
dosette_boxes_ccg.head()


,Items,sevendayitems,percent_seven_days
PCO_Name,,,
"AIREDALE, WHARFEDALE AND CRAVEN CCG",5081,600,11.808699
ASHFORD CCG,3521,119,3.379722
BARKING & DAGENHAM CCG,6031,2999,49.726414
BARNET CCG,9132,3235,35.424880
BARNSLEY CCG,14304,4509,31.522651


In [8]:
dosette_boxes_ccg.describe()

,Items,sevendayitems,percent_seven_days
count,205.000000,205.000000,205.000000
mean,8927.912195,1842.439024,21.583293
std,6438.558804,2114.781009,18.058489
min,2.000000,1.000000,0.862069
25%,4998.000000,390.000000,7.663965
50%,7366.000000,1074.000000,14.285714
75%,10569.000000,2729.000000,33.333333
max,41798.000000,11275.000000,100.000000


In [16]:
dosette_boxes_ccg.sort_values('percent_seven_days', ascending=False).head(10)

,Items,sevendayitems,percent_seven_days
PCO_Name,,,
VOCARE,2,2,100.000000
TOWER HAMLETS CCG,9165,6677,72.853246
LAMBETH CCG,8002,5545,69.295176
WANDSWORTH CCG,8765,5876,67.039361
OXFORD HEALTH NHS FOUNDATION TRUST,3,2,66.666667
SOUTHWARK CCG,6906,4581,66.333623
HAMMERSMITH AND FULHAM CCG,4788,3138,65.538847
EALING CCG,13363,7754,58.025892
HARINGEY CCG,7275,4126,56.714777


In [9]:
dosette_boxes_ccg.to_csv('aspirin.csv')

In [10]:
#work out english total
eng_total2 = dosette_boxes_ccg.sum()[['Items','sevendayitems']]
eng_total2

Items            1830222.0
sevendayitems     377700.0
dtype: float64

circa 21% of Aspirin 75mg items are written as a seven day prescription.

 ## Now we take a different approach looking at the proportion of tablets that are written as part of a seven day prescription

In [11]:
sql = '''
SELECT
  PCO_Name,
  PCO_Code,
  SUM(CASE
      WHEN quantity=7 THEN quantity*items
      ELSE 0
  END) AS _7_day_QI,
  SUM(quantity*items) AS all_QI,
  IEEE_DIVIDE(SUM(CASE
      WHEN quantity=7 THEN quantity*items
      ELSE 0
  END),
      SUM(quantity*items)) AS percent_7_day_QI
FROM
ebmdatalab.tmp_eu.raw_prescribing_data_2019_04 AS presc
INNER JOIN
  hscic.ccgs AS ccg
ON
  SUBSTR(presc.pco_code,0,3)=ccg.code
WHERE
    (bnf_code LIKE '0209000A0%AB' ##dispersible 75mg
    OR bnf_code LIKE '0209000A0%AJ' ##normal 75mg tabs
    OR bnf_code LIKE '0209000A0%AK') ##enteric coated 75mg
    AND ccg.org_type = 'CCG'
GROUP BY
  PCO_Name,
  PCO_Code
ORDER BY
  percent_7_day_QI DESC 
'''

df2_aspirin = bq.cached_read(sql, csv_path='aspirintwo_sevendays.csv')
df2_aspirin.rename(columns={'PCO_Code':'pct'}, inplace=True)
df2_aspirin.head(5)

C:\Users\bmackenna\Anaconda3\lib\site-packages\pandas\io\gbq.py:149: FutureWarning: verbose is deprecated and will be removed in a future version. Set logging level in order to vary verbosity
  credentials=credentials, verbose=verbose, private_key=private_key)


,PCO_Name,pct,_7_day_QI,all_QI,percent_7_day_QI
0,TOWER HAMLETS CCG,08V00,46739,167342,0.279302
1,LAMBETH CCG,08K00,38815,169609,0.228850
2,WANDSWORTH CCG,08X00,41188,186930,0.220339
3,HAMMERSMITH AND FULHAM CCG,08C00,21966,106866,0.205547
4,SOUTHWARK CCG,08Q00,32109,159797,0.200936


In [12]:
df2_aspirin.describe()

,_7_day_QI,all_QI,percent_7_day_QI
count,191.000000,1.910000e+02,191.000000
mean,13864.691099,2.924456e+05,0.050690
std,14941.107554,1.916826e+05,0.050302
min,154.000000,7.251000e+04,0.002124
25%,3402.000000,1.755030e+05,0.015340
50%,8225.000000,2.391130e+05,0.029248
75%,20254.500000,3.390955e+05,0.079130
max,78925.000000,1.340798e+06,0.279302


### Other possible drugs - atorvastatin and simvastatin

In [4]:
sql = '''
SELECT
  PCO_Name,
  PCO_Code,
  SUM(CASE
      WHEN quantity=7 THEN quantity*items
      ELSE 0
  END) AS _7_day_QI,
  SUM(quantity*items) AS all_QI,
  IEEE_DIVIDE(SUM(CASE
      WHEN quantity=7 THEN quantity*items
      ELSE 0
  END),
      SUM(quantity*items)) AS percent_7_day_QI
FROM
  tmp_eu.raw_prescribing_data_2019_04 AS presc
INNER JOIN
  hscic.ccgs AS ccg
ON
  SUBSTR(presc.pco_code,0,3)=ccg.code
WHERE
(bnf_description LIKE 'Simvastatin_Tab%' or bnf_description LIKE 'Atorvastatin_Tab%')
    AND ccg.org_type = 'CCG'
GROUP BY
  PCO_Name,
  PCO_Code
ORDER BY
  percent_7_day_QI DESC
  '''

df2_statin = bq.cached_read(sql, csv_path='statin_sevendays.csv')
df2_statin.rename(columns={'PCO_Code':'pct'}, inplace=True)
df2_statin.head(5)

C:\Users\racro\Anaconda3\lib\site-packages\pandas\io\gbq.py:149: FutureWarning: verbose is deprecated and will be removed in a future version. Set logging level in order to vary verbosity
  credentials=credentials, verbose=verbose, private_key=private_key)


,PCO_Name,pct,_7_day_QI,all_QI,percent_7_day_QI
0,TOWER HAMLETS CCG,08V00,137452,725492,0.189460
1,WANDSWORTH CCG,08X00,97720,610329,0.160110
2,LAMBETH CCG,08K00,102655,668041,0.153666
3,SUNDERLAND CCG,00P00,168679,1220536,0.138201
4,SOUTHWARK CCG,08Q00,84756,614439,0.137940


### Ramipril

In [6]:
sql = '''
SELECT
  PCO_Name,
  PCO_Code,
  SUM(CASE
      WHEN quantity=7 THEN quantity*items
      ELSE 0
  END) AS _7_day_QI,
  SUM(quantity*items) AS all_QI,
  IEEE_DIVIDE(SUM(CASE
      WHEN quantity=7 THEN quantity*items
      ELSE 0
  END),
      SUM(quantity*items)) AS percent_7_day_QI
FROM
  tmp_eu.raw_prescribing_data_2019_04 AS presc
INNER JOIN
  hscic.ccgs AS ccg
ON
  SUBSTR(presc.pco_code,0,3)=ccg.code
WHERE
(bnf_description LIKE 'Rampril_Tab%' or bnf_description LIKE 'Ramipril_Cap%')
    AND ccg.org_type = 'CCG'
GROUP BY
  PCO_Name,
  PCO_Code
ORDER BY
  percent_7_day_QI DESC
    '''

df2_ramipril = bq.cached_read(sql, csv_path='ramipril_sevendays.csv')
df2_ramipril.rename(columns={'PCO_Code':'pct'}, inplace=True)
df2_ramipril.head(5)

,PCO_Name,pct,_7_day_QI,all_QI,percent_7_day_QI
0,TOWER HAMLETS CCG,08V00,42077,254273,0.165480
1,SUNDERLAND CCG,00P00,26572,200406,0.132591
2,HAMMERSMITH AND FULHAM CCG,08C00,14091,113366,0.124297
3,WANDSWORTH CCG,08X00,33397,275637,0.121163
4,LAMBETH CCG,08K00,39291,328798,0.119499
